In [1]:
from arcgishub import hub

In [3]:
myHub = hub.Hub("https://dcdev.maps.arcgis.com", 'mmajumdar_dcdev')

Enter password: ········


#### Fetching a single initiative

In [4]:
walkability_init = myHub.initiatives.get('07fad5be49da45bea02c8f5b0876be24')

In [5]:
walkability_init.item

<Item title:"Walkability Assessment " type:Hub Initiative owner:mmajumdar_dcdev>

In [6]:
walkability_init.siteUrl

'http://walkability-assessment-dcdev.hub.arcgis.com'

#### Adding new initiaitve

In [7]:
myInitiative = myHub.initiatives.add(title='Python add initiative')

In [8]:
myInitiative.item

<Item title:"Python add initiative" type:Hub Initiative owner:mmajumdar_dcdev>

In [ ]:
myIn

#### Updating initiative

In [9]:
myInitiative_data = myInitiative.get_data()

In [10]:
myInitiative_data['snippet'] = 'Create your own initiative by combining existing applications with a custom site'

In [11]:
myInitiative.update(initiative_properties=myInitiative_data)

True

In [12]:
myInitiative.item

<Item title:"Python add initiative" type:Hub Initiative owner:mmajumdar_dcdev>

Refer the [item properties table](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.gis.toc.html#arcgis.gis.Item.update) to learn more about `initiative_properties` that can be updated using this function

#### Searching for initiatives

In [13]:
myHub.initiatives.search(owner='mmajumdar_dcdev', tags='Hub')

[<Initiative title:"Vision Zero ManushiM" owner:mmajumdar_dcdev>,
 <Initiative title:"Vision Zero with Collision Lens App Florida" owner:mmajumdar_dcdev>,
 <Initiative title:"Place Boundary analysis - DC" owner:mmajumdar_dcdev>,
 <Initiative title:"Vision Zero Manushi" owner:mmajumdar_dcdev>,
 <Initiative title:"Vision Zero - DevSummit" owner:mmajumdar_dcdev>,
 <Initiative title:"Vision Zero 1" owner:mmajumdar_dcdev>]

#### Delete initiative

In [14]:
myInitiative.delete()

True